<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/spell_check_%EC%9D%BC%EB%B6%80_%EC%A6%9D%EB%B6%84_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A1%9C_%EB%8C%80%EB%B6%84%EB%A5%98_%EC%98%88%EC%B8%A1%ED%95%98%EA%B8%B0(%EC%B5%9C%EC%A2%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 11.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595729 sha256=187363b0ff1ab6957a6c5d50c1ab9f012b861324b3ed0a8f71b5e09f466751af
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 11.9 MB/s 
     |████████████████████████████████| 769 kB 13.3 MB/s 
     |████████████████████████████████| 3.0 MB 59.4 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-f8885mnq
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_A_inc.csv   df_M_inc.csv	 labels.zip
 A_inc_model.pt		 df_B_inc.csv   df_N_inc.csv	'My Drive'
 B_inc_model.pt		 df_D_inc.csv   df_O_inc.csv	 spell_check
 classes.txt		 df_E_inc.csv   df_Q_inc.csv	 val.txt
'Colab Notebooks'	 df_J_inc.csv   df_R_inc.csv
'답안 작성용 파일.csv'	 df_K_inc.csv   E_inc_model.pt


In [4]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'모델 개발용 자료'.zip -d /content/dataset
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/모델 개발용 자료.zip
2. 모델개발용자료.txt:  mismatching "local" filename (2. ыкиыН╕ъ░Ьы░ЬьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/2. 모델개발용자료.txt  
Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [5]:
import pandas as pd
df_A=pd.read_csv('/content/gdrive/MyDrive/My Drive/df_A_inc.csv') #증분데이터 사용
df_B=pd.read_csv('/content/gdrive/MyDrive/My Drive/df_B_inc.csv') #증분데이터 사용
df_C=pd.read_csv('/content/gdrive/MyDrive/spell_check/C_spellcheck.csv')
df_D=pd.read_csv('/content/gdrive/MyDrive/My Drive/df_D_inc.csv') #증분데이터 사용
df_E=pd.read_csv('/content/gdrive/MyDrive/My Drive/df_E_inc.csv') #증분데이터 사용
df_F=pd.read_csv('/content/gdrive/MyDrive/spell_check/F_spell_check.csv')
df_G=pd.read_csv('/content/gdrive/MyDrive/spell_check/G_spell_check.csv')
df_H=pd.read_csv('/content/gdrive/MyDrive/spell_check/H_spellcheck.csv')
df_I=pd.read_csv('/content/gdrive/MyDrive/spell_check/I_spell_check.csv')
df_J=pd.read_csv('/content/gdrive/MyDrive/spell_check/J_spellcheck.csv')
df_K=pd.read_csv('/content/gdrive/MyDrive/spell_check/K_spellcheck.csv')
df_L=pd.read_csv('/content/gdrive/MyDrive/spell_check/L_spell_check.csv')
df_M=pd.read_csv('/content/gdrive/MyDrive/spell_check/M_spell_check.csv')
df_N=pd.read_csv('/content/gdrive/MyDrive/spell_check/N_spell_check.csv')
df_O=pd.read_csv('/content/gdrive/MyDrive/My Drive/df_O_inc.csv') #증분데이터 사용
df_P=pd.read_csv('/content/gdrive/MyDrive/spell_check/P_spell_check.csv')
df_Q=pd.read_csv('/content/gdrive/MyDrive/spell_check/Q_spellcheck.csv')
df_R=pd.read_csv('/content/gdrive/MyDrive/spell_check/R_spellcheck.csv')
df_S=pd.read_csv('/content/gdrive/MyDrive/spell_check/S_spellcheck.csv')

In [6]:
df=pd.concat([df_A, df_B, df_C, df_D, df_E, df_F, df_G, df_H, df_I, df_J, df_K, df_L, df_M, df_N, df_O, df_P, df_Q, df_R, df_S])

In [7]:
df['digit_1'].value_counts()

G    246472
I    187425
C    105192
S    100396
H     98038
P     46610
L     40140
Q     36087
F     35050
R     29751
M     28434
N     17701
A     13275
J     10862
E     10428
K     10378
O      6380
B      5847
D      1968
Name: digit_1, dtype: int64

**데이터 전처리**

In [8]:
df.loc[(df['digit_1'] == "A"), 'digit_1'] = 0  
df.loc[(df['digit_1'] == "B"), 'digit_1'] = 1  
df.loc[(df['digit_1'] == "C"), 'digit_1'] = 2  
df.loc[(df['digit_1'] == "D"), 'digit_1'] = 3  
df.loc[(df['digit_1'] == "E"), 'digit_1'] = 4  
df.loc[(df['digit_1'] == "F"), 'digit_1'] = 5  
df.loc[(df['digit_1'] == "G"), 'digit_1'] = 6  
df.loc[(df['digit_1'] == "H"), 'digit_1'] = 7  
df.loc[(df['digit_1'] == "I"), 'digit_1'] = 8  
df.loc[(df['digit_1'] == "J"), 'digit_1'] = 9  
df.loc[(df['digit_1'] == "K"), 'digit_1'] = 10  
df.loc[(df['digit_1'] == "L"), 'digit_1'] = 11 
df.loc[(df['digit_1'] == "M"), 'digit_1'] = 12  
df.loc[(df['digit_1'] == "N"), 'digit_1'] = 13  
df.loc[(df['digit_1'] == "O"), 'digit_1'] = 14  
df.loc[(df['digit_1'] == "P"), 'digit_1'] = 15  
df.loc[(df['digit_1'] == "Q"), 'digit_1'] = 16 
df.loc[(df['digit_1'] == "R"), 'digit_1'] = 17  
df.loc[(df['digit_1'] == "S"), 'digit_1'] = 18  

In [9]:
data_list = []
for q, label in zip(df['text'], df['digit_1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['벼 맥류 콩류 잡곡류 종자 생산 보급', '0'], ['벼 모판 생산 농가 출하', '0'], ['사업장에서 오이. 고추를 재배 오이. 고추를 판매', '0'], ['볍씨 발아 모종', '0'], ['  벼 재배', '0']]


In [10]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

Kobert 입력데이터 만들기

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [14]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

Kobert 학습 모델 만들기

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=19,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [18]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/14491 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.0587663650512695 train acc 0.03125
epoch 1 batch id 201 loss 2.7217934131622314 train acc 0.1052549751243781
epoch 1 batch id 401 loss 2.047919750213623 train acc 0.24575280548628428
epoch 1 batch id 601 loss 1.4881665706634521 train acc 0.3685784109816972
epoch 1 batch id 801 loss 0.954892098903656 train acc 0.46900358926342073
epoch 1 batch id 1001 loss 0.6700411438941956 train acc 0.5433160589410589
epoch 1 batch id 1201 loss 0.38813525438308716 train acc 0.5995134263114071
epoch 1 batch id 1401 loss 0.46311089396476746 train acc 0.6435135617416131
epoch 1 batch id 1601 loss 0.46326014399528503 train acc 0.6792239225484072
epoch 1 batch id 1801 loss 0.3387804627418518 train acc 0.7082350083287062
epoch 1 batch id 2001 loss 0.18315458297729492 train acc 0.7321651674162919
epoch 1 batch id 2201 loss 0.20839367806911469 train acc 0.7523355860972285
epoch 1 batch id 2401 loss 0.17604488134384155 train acc 0.769152176176593
epoch 1 batch id 2601 loss 0.261954486

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1611 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9815914028553694


  0%|          | 0/14491 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0029182513244450092 train acc 1.0
epoch 5 batch id 201 loss 0.08408263325691223 train acc 0.992070895522388
epoch 5 batch id 401 loss 0.019605141133069992 train acc 0.9921290523690773
epoch 5 batch id 601 loss 0.008431264199316502 train acc 0.9918365224625624
epoch 5 batch id 801 loss 0.017202377319335938 train acc 0.9914169787765293
epoch 5 batch id 1001 loss 0.07642314583063126 train acc 0.9913367882117882
epoch 5 batch id 1201 loss 0.023335466161370277 train acc 0.9914264154870941
epoch 5 batch id 1401 loss 0.04355621337890625 train acc 0.9914793004996431
epoch 5 batch id 1601 loss 0.0056004212237894535 train acc 0.9915872891942535
epoch 5 batch id 1801 loss 0.005468948744237423 train acc 0.9916452665186007
epoch 5 batch id 2001 loss 0.009797507897019386 train acc 0.9918634432783608
epoch 5 batch id 2201 loss 0.006222181022167206 train acc 0.9919922762380736
epoch 5 batch id 2401 loss 0.001511429320089519 train acc 0.9920215535193669
epoch 5 batch id 2601 l

  0%|          | 0/1611 [00:00<?, ?it/s]

epoch 5 test acc 0.9820084574798262


a,b,d,e,o만 증분데이터 acc 0.982008457

In [19]:
#학습 모델 저장
torch.save(model, 'dabunryu_inc_model.pt')

In [20]:
model_save_name='dabunru_inc_model.pt'
path=F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model.state_dict(), path)